# Planning
#### Chapters 10-11
----

This notebook serves as supporting material for topics covered in **Chapter 10 - Classical Planning** and **Chapter 11 - Planning and Acting in the Real World** from the book *[Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu)*. 
This notebook uses implementations from the [planning.py](https://github.com/aimacode/aima-python/blob/master/planning.py) module. 
See the [intro notebook](https://github.com/aimacode/aima-python/blob/master/intro.ipynb) for instructions.

We'll start by looking at `PlanningProblem` and `Action` data types for defining problems and actions. 
Then, we will see how to use them by trying to plan a trip from *Sibiu* to *Bucharest* across the familiar map of Romania, from [search.ipynb](https://github.com/aimacode/aima-python/blob/master/search.ipynb) 
followed by some common planning problems and methods of solving them.

Let's start by importing everything from the planning module.

In [1]:
from planning import *
from notebook import psource

## CONTENTS

**Classical Planning**
- PlanningProblem
- Action
- Planning Problems
    * Air cargo problem
    * Spare tire problem
    * Three block tower problem
    * Shopping Problem
    * Socks and shoes problem
    * Cake problem
- Solving Planning Problems
    * GraphPlan
    * Linearize
    * PartialOrderPlanner
<br>

**Planning in the real world**
- Problem
- HLA
- Planning Problems
    * Job shop problem
    * Double tennis problem
- Solving Planning Problems
    * Hierarchical Search
    * Angelic Search

## PlanningProblem

PDDL stands for Planning Domain Definition Language.
The `PlanningProblem` class is used to represent planning problems in this module. The following attributes are essential to be able to define a problem:
* an initial state
* a set of goals
* a set of viable actions that can be executed in the search space of the problem

View the source to see how the Python code tries to realise these.

In [2]:
psource(PlanningProblem)

The `init` attribute is an expression that forms the initial knowledge base for the problem.
<br>
The `goals` attribute is an expression that indicates the goals to be reached by the problem.
<br>
Lastly, `actions` contains a list of `Action` objects that may be executed in the search space of the problem.
<br>
The `goal_test` method checks if the goal has been reached.
<br>
The `act` method acts out the given action and updates the current state.
<br>


## ACTION

To be able to model a planning problem properly, it is essential to be able to represent an Action. Each action we model requires at least three things:
* preconditions that the action must meet
* the effects of executing the action
* some expression that represents the action

The module models actions using the `Action` class

In [3]:
psource(Action)

This class represents an action given the expression, the preconditions and its effects. 
A list `precond` stores the preconditions of the action and a list `effect` stores its effects.
Negative preconditions and effects are input using a `~` symbol before the clause, which are internally prefixed with a `Not` to make it easier to work with.
For example, the negation of `At(obj, loc)` will be input as `~At(obj, loc)` and internally represented as `NotAt(obj, loc)`. 
This equivalently creates a new clause for each negative literal, removing the hassle of maintaining two separate knowledge bases.
This greatly simplifies algorithms like `GraphPlan` as we will see later.
The `convert` method takes an input string, parses it, removes conjunctions if any and returns a list of `Expr` objects.
The `check_precond` method checks if the preconditions for that action are valid, given a `kb`.
The `act` method carries out the action on the given knowledge base.

Now lets try to define a planning problem using these tools. Since we already know about the map of Romania, lets see if we can plan a trip across a simplified map of Romania.

Here is our simplified map definition:

In [4]:
from utils import *
# this imports the required expr so we can create our knowledge base

knowledge_base = [
    expr("Connected(Bucharest,Pitesti)"),
    expr("Connected(Pitesti,Rimnicu)"),
    expr("Connected(Rimnicu,Sibiu)"),
    expr("Connected(Sibiu,Fagaras)"),
    expr("Connected(Fagaras,Bucharest)"),
    expr("Connected(Pitesti,Craiova)"),
    expr("Connected(Craiova,Rimnicu)")
    ]

Let us add some logic propositions to complete our knowledge about travelling around the map. These are the typical symmetry and transitivity properties of connections on a map. We can now be sure that our `knowledge_base` understands what it truly means for two locations to be connected in the sense usually meant by humans when we use the term.

Let's also add our starting location - *Sibiu* to the map.

In [5]:
knowledge_base.extend([
     expr("Connected(x,y) ==> Connected(y,x)"),
     expr("Connected(x,y) & Connected(y,z) ==> Connected(x,z)"),
     expr("At(Sibiu)")
    ])

We now have a complete knowledge base, which can be seen like this:

In [6]:
knowledge_base

[Connected(Bucharest, Pitesti),
 Connected(Pitesti, Rimnicu),
 Connected(Rimnicu, Sibiu),
 Connected(Sibiu, Fagaras),
 Connected(Fagaras, Bucharest),
 Connected(Pitesti, Craiova),
 Connected(Craiova, Rimnicu),
 (Connected(x, y) ==> Connected(y, x)),
 ((Connected(x, y) & Connected(y, z)) ==> Connected(x, z)),
 At(Sibiu)]

We now define possible actions to our problem. We know that we can drive between any connected places. But, as is evident from [this](https://en.wikipedia.org/wiki/List_of_airports_in_Romania) list of Romanian airports, we can also fly directly between Sibiu, Bucharest, and Craiova.

We can define these flight actions like this:

In [7]:
#Sibiu to Bucharest
precond = 'At(Sibiu)'
effect = 'At(Bucharest) & ~At(Sibiu)'
fly_s_b = Action('Fly(Sibiu, Bucharest)', precond, effect)

#Bucharest to Sibiu
precond = 'At(Bucharest)'
effect = 'At(Sibiu) & ~At(Bucharest)'
fly_b_s = Action('Fly(Bucharest, Sibiu)', precond, effect)

#Sibiu to Craiova
precond = 'At(Sibiu)'
effect = 'At(Craiova) & ~At(Sibiu)'
fly_s_c = Action('Fly(Sibiu, Craiova)', precond, effect)

#Craiova to Sibiu
precond = 'At(Craiova)'
effect = 'At(Sibiu) & ~At(Craiova)'
fly_c_s = Action('Fly(Craiova, Sibiu)', precond, effect)

#Bucharest to Craiova
precond = 'At(Bucharest)'
effect = 'At(Craiova) & ~At(Bucharest)'
fly_b_c = Action('Fly(Bucharest, Craiova)', precond, effect)

#Craiova to Bucharest
precond = 'At(Craiova)'
effect = 'At(Bucharest) & ~At(Craiova)'
fly_c_b = Action('Fly(Craiova, Bucharest)', precond, effect)

And the drive actions like this.

In [8]:
#Drive
precond = 'At(x)'
effect = 'At(y) & ~At(x)'
drive = Action('Drive(x, y)', precond, effect)

Our goal is defined as

In [9]:
goals = 'At(Bucharest)'

Finally, we can define a a function that will tell us when we have reached our destination, Bucharest.

In [10]:
def goal_test(kb):
    return kb.ask(expr('At(Bucharest)'))

Thus, with all the components in place, we can define the planning problem.

In [11]:
prob = PlanningProblem(knowledge_base, goals, [fly_s_b, fly_b_s, fly_s_c, fly_c_s, fly_b_c, fly_c_b, drive])

## PLANNING PROBLEMS
---

## Air Cargo Problem

In the Air Cargo problem, we start with cargo at two airports, SFO and JFK. Our goal is to send each cargo to the other airport. We have two airplanes to help us accomplish the task. 
The problem can be defined with three actions: Load, Unload and Fly. 
Let us look how the `air_cargo` problem has been defined in the module. 

In [12]:
psource(air_cargo)

**At(c, a):** The cargo **'c'** is at airport **'a'**.

**~At(c, a):** The cargo **'c'** is _not_ at airport **'a'**.

**In(c, p):** Cargo **'c'** is in plane **'p'**.

**~In(c, p):** Cargo **'c'** is _not_ in plane **'p'**.

**Cargo(c):** Declare **'c'** as cargo.

**Plane(p):** Declare **'p'** as plane.

**Airport(a):** Declare **'a'** as airport.



In the `initial_state`, we have cargo C1, plane P1 at airport SFO and cargo C2, plane P2 at airport JFK. 
Our goal state is to have cargo C1 at airport JFK and cargo C2 at airport SFO. We will discuss on how to achieve this. Let us now define an object of the `air_cargo` problem:

In [13]:
airCargo = air_cargo()

Before taking any actions, we will check if `airCargo` has reached its goal:

In [14]:
print(airCargo.goal_test())

False


It returns False because the goal state is not yet reached. Now, we define the sequence of actions that it should take in order to achieve the goal.
The actions are then carried out on the `airCargo` PlanningProblem.

The actions available to us are the following: Load, Unload, Fly

**Load(c, p, a):** Load cargo **'c'** into plane **'p'** from airport **'a'**.

**Fly(p, f, t):** Fly the plane **'p'** from airport **'f'** to airport **'t'**.

**Unload(c, p, a):** Unload cargo **'c'** from plane **'p'** to airport **'a'**.

This problem can have multiple valid solutions.
One such solution is shown below.

In [15]:
solution = [expr("Load(C1 , P1, SFO)"),
            expr("Fly(P1, SFO, JFK)"),
            expr("Unload(C1, P1, JFK)"),
            expr("Load(C2, P2, JFK)"),
            expr("Fly(P2, JFK, SFO)"),
            expr("Unload (C2, P2, SFO)")] 

for action in solution:
    airCargo.act(action)

As the `airCargo` has taken all the steps it needed in order to achieve the goal, we can now check if it has acheived its goal:

In [16]:
print(airCargo.goal_test())

True


It has now achieved its goal.

## The Spare Tire Problem

Let's consider the problem of changing a flat tire of a car. 
The goal is to mount a spare tire onto the car's axle, given that we have a flat tire on the axle and a spare tire in the trunk. 

In [17]:
psource(spare_tire)

**At(obj, loc):** object **'obj'** is at location **'loc'**.

**~At(obj, loc):** object **'obj'** is _not_ at location **'loc'**.

**Tire(t):** Declare a tire of type **'t'**.

Let us now define an object of `spare_tire` problem:

In [18]:
spareTire = spare_tire()

Before taking any actions, we will check if `spare_tire` has reached its goal:

In [19]:
print(spareTire.goal_test())

False


As we can see, it hasn't completed the goal. 
We now define a possible solution that can help us reach the goal of having a spare tire mounted onto the car's axle. 
The actions are then carried out on the `spareTire` PlanningProblem.

The actions available to us are the following: Remove, PutOn

**Remove(obj, loc):** Remove the tire **'obj'** from the location **'loc'**.

**PutOn(t, Axle):** Attach the tire **'t'** on the Axle.

**LeaveOvernight():** We live in a particularly bad neighborhood and all tires, flat or not, are stolen if we leave them overnight.



In [20]:
solution = [expr("Remove(Flat, Axle)"),
            expr("Remove(Spare, Trunk)"),
            expr("PutOn(Spare, Axle)")]

for action in solution:
    spareTire.act(action)

In [21]:
print(spareTire.goal_test())

True


This is a valid solution.
<br>
Another possible solution is

In [22]:
spareTire = spare_tire()

solution = [expr('Remove(Spare, Trunk)'),
            expr('Remove(Flat, Axle)'),
            expr('PutOn(Spare, Axle)')]

for action in solution:
    spareTire.act(action)

In [23]:
print(spareTire.goal_test())

True


Notice that both solutions work, which means that the problem can be solved irrespective of the order in which the `Remove` actions take place, as long as both `Remove` actions take place before the `PutOn` action.

We have successfully mounted a spare tire onto the axle.

## Three Block Tower Problem

This problem's domain consists of a set of cube-shaped blocks sitting on a table. 
The blocks can be stacked, but only one block can fit directly on top of another.
A robot arm can pick up a block and move it to another position, either on the table or on top of another block. 
The arm can pick up only one block at a time, so it cannot pick up a block that has another one on it. 
The goal will always be to build one or more stacks of blocks. 
In our case, we consider only three blocks.
The particular configuration we will use is called the Sussman anomaly after Prof. Gerry Sussman.

Let's take a look at the definition of `three_block_tower()` in the module.

In [24]:
psource(three_block_tower)

**On(b, x):** The block **'b'** is on **'x'**. **'x'** can be a table or a block.

**~On(b, x):** The block **'b'** is _not_ on **'x'**. **'x'** can be a table or a block.

**Block(b):** Declares **'b'** as a block.

**Clear(x):** To indicate that there is nothing on **'x'** and it is free to be moved around.

**~Clear(x):** To indicate that there is something on **'x'** and it cannot be moved.
 
 Let us now define an object of `three_block_tower` problem:

In [25]:
threeBlockTower = three_block_tower()

Before taking any actions, we will check if `threeBlockTower` has reached its goal:

In [26]:
print(threeBlockTower.goal_test())

False


As we can see, it hasn't completed the goal. 
We now define a sequence of actions that can stack three blocks in the required order. 
The actions are then carried out on the `threeBlockTower` PlanningProblem.

The actions available to us are the following: MoveToTable, Move

**MoveToTable(b, x): ** Move box **'b'** stacked on **'x'** to the table, given that box **'b'** is clear.

**Move(b, x, y): ** Move box **'b'** stacked on **'x'** to the top of **'y'**, given that both **'b'** and **'y'** are clear.


In [27]:
solution = [expr("MoveToTable(C, A)"),
            expr("Move(B, Table, C)"),
            expr("Move(A, Table, B)")]

for action in solution:
    threeBlockTower.act(action)

As the `three_block_tower` has taken all the steps it needed in order to achieve the goal, we can now check if it has acheived its goal.

In [28]:
print(threeBlockTower.goal_test())

True


It has now successfully achieved its goal i.e, to build a stack of three blocks in the specified order.

The `three_block_tower` problem can also be defined in simpler terms using just two actions `ToTable(x, y)` and `FromTable(x, y)`.
The underlying problem remains the same however, stacking up three blocks in a certain configuration given a particular starting state.
Let's have a look at the alternative definition.

In [29]:
psource(simple_blocks_world)

**On(x, y):** The block **'x'** is on **'y'**. Both **'x'** and **'y'** have to be blocks.

**~On(x, y):** The block **'x'** is _not_ on **'y'**. Both **'x'** and **'y'** have to be blocks.

**OnTable(x):** The block **'x'** is on the table.

**~OnTable(x):** The block **'x'** is _not_ on the table.

**Clear(x):** To indicate that there is nothing on **'x'** and it is free to be moved around.

**~Clear(x):** To indicate that there is something on **'x'** and it cannot be moved.

Let's now define a `simple_blocks_world` prolem.

In [30]:
simpleBlocksWorld = simple_blocks_world()

Before taking any actions, we will see if `simple_bw` has reached its goal.

In [31]:
simpleBlocksWorld.goal_test()

False

As we can see, it hasn't completed the goal. 
We now define a sequence of actions that can stack three blocks in the required order. 
The actions are then carried out on the `simple_bw` PlanningProblem.

The actions available to us are the following: MoveToTable, Move

**ToTable(x, y): ** Move box **'x'** stacked on **'y'** to the table, given that box **'y'** is clear.

**FromTable(x, y): ** Move box **'x'** from wherever it is, to the top of **'y'**, given that both **'x'** and **'y'** are clear.


In [32]:
solution = [expr('ToTable(A, B)'),
            expr('FromTable(B, A)'),
            expr('FromTable(C, B)')]

for action in solution:
    simpleBlocksWorld.act(action)

As the `three_block_tower` has taken all the steps it needed in order to achieve the goal, we can now check if it has acheived its goal.

In [33]:
print(simpleBlocksWorld.goal_test())

True


It has now successfully achieved its goal i.e, to build a stack of three blocks in the specified order.

## Shopping Problem

This problem requires us to acquire a carton of milk, a banana and a drill.
Initially, we start from home and it is known to us that milk and bananas are available in the supermarket and the hardware store sells drills.
Let's take a look at the definition of the `shopping_problem` in the module.

In [34]:
psource(shopping_problem)

**At(x):** Indicates that we are currently at **'x'** where **'x'** can be Home, SM (supermarket) or HW (Hardware store).

**~At(x):** Indicates that we are currently _not_ at **'x'**.

**Sells(s, x):** Indicates that item **'x'** can be bought from store **'s'**.

**Have(x):** Indicates that we possess the item **'x'**.

In [35]:
shoppingProblem = shopping_problem()

Let's first check whether the goal state Have(Milk), Have(Banana), Have(Drill) is reached or not.

In [36]:
print(shoppingProblem.goal_test())

False


Let's look at the possible actions

**Buy(x, store):** Buy an item **'x'** from a **'store'** given that the **'store'** sells **'x'**.

**Go(x, y):** Go to destination **'y'** starting from source **'x'**.

We now define a valid solution that will help us reach the goal.
The sequence of actions will then be carried out onto the `shoppingProblem` PlanningProblem.

In [37]:
solution = [expr('Go(Home, SM)'),
            expr('Buy(Milk, SM)'),
            expr('Buy(Banana, SM)'),
            expr('Go(SM, HW)'),
            expr('Buy(Drill, HW)')]

for action in solution:
    shoppingProblem.act(action)

We have taken the steps required to acquire all the stuff we need. 
Let's see if we have reached our goal.

In [38]:
shoppingProblem.goal_test()

True

It has now successfully achieved the goal.

## Socks and Shoes

This is a simple problem of putting on a pair of socks and shoes.
The problem is defined in the module as given below.

In [39]:
psource(socks_and_shoes)

**LeftSockOn:** Indicates that we have already put on the left sock.

**RightSockOn:** Indicates that we have already put on the right sock.

**LeftShoeOn:** Indicates that we have already put on the left shoe.

**RightShoeOn:** Indicates that we have already put on the right shoe.


In [40]:
socksShoes = socks_and_shoes()

Let's first check whether the goal state is reached or not.

In [41]:
socksShoes.goal_test()

False

As the goal state isn't reached, we will define a sequence of actions that might help us achieve the goal.
These actions will then be acted upon the `socksShoes` PlanningProblem to check if the goal state is reached.

In [42]:
solution = [expr('RightSock'),
            expr('RightShoe'),
            expr('LeftSock'),
            expr('LeftShoe')]

In [43]:
for action in solution:
    socksShoes.act(action)
    
socksShoes.goal_test()

True

We have reached our goal.

## Cake Problem

This problem requires us to reach the state of having a cake and having eaten a cake simlutaneously, given a single cake.
Let's first take a look at the definition of the `have_cake_and_eat_cake_too` problem in the module.

In [44]:
psource(have_cake_and_eat_cake_too)

Since this problem doesn't involve variables, states can be considered similar to symbols in propositional logic.

**Have(Cake):** Declares that we have a **'Cake'**.

**~Have(Cake):** Declares that we _don't_ have a **'Cake'**.

In [45]:
cakeProblem = have_cake_and_eat_cake_too()

First let us check whether the goal state 'Have(Cake)' and 'Eaten(Cake)' are reached or not.

In [46]:
print(cakeProblem.goal_test())

False


Let us look at the possible actions.

**Bake(x):** To bake **' x '**.

**Eat(x):** To eat **' x '**.

We now define a valid solution that can help us reach the goal.
The sequence of actions will then be acted upon the `cakeProblem` PlanningProblem.

In [47]:
solution = [expr("Eat(Cake)"),
            expr("Bake(Cake)")]

for action in solution:
    cakeProblem.act(action)

Now we have made actions to bake the cake and eat the cake. Let us check if we have reached the goal.

In [48]:
print(cakeProblem.goal_test())

True


It has now successfully achieved its goal i.e, to have and eat the cake.

One might wonder if the order of the actions matters for this problem.
Let's see for ourselves.

In [49]:
cakeProblem = have_cake_and_eat_cake_too()

solution = [expr('Bake(Cake)'),
            expr('Eat(Cake)')]

for action in solution:
    cakeProblem.act(action)

Exception: Action 'Bake(Cake)' pre-conditions not satisfied

It raises an exception.
Indeed, according to the problem, we cannot bake a cake if we already have one.
In planning terms, '~Have(Cake)' is a precondition to the action 'Bake(Cake)'.
Hence, this solution is invalid.

## SOLVING PLANNING PROBLEMS
----
### GRAPHPLAN
<br>
The GraphPlan algorithm is a popular method of solving classical planning problems.
Before we get into the details of the algorithm, let's look at a special data structure called **planning graph**, used to give better heuristic estimates and plays a key role in the GraphPlan algorithm.

### Planning Graph
A planning graph is a directed graph organized into levels. 
Each level contains information about the current state of the knowledge base and the possible state-action links to and from that level.
The first level contains the initial state with nodes representing each fluent that holds in that level.
This level has state-action links linking each state to valid actions in that state.
Each action is linked to all its preconditions and its effect states.
Based on these effects, the next level is constructed.
The next level contains similarly structured information about the next state.
In this way, the graph is expanded using state-action links till we reach a state where all the required goals hold true simultaneously.
We can say that we have reached our goal if none of the goal states in the current level are mutually exclusive.
This will be explained in detail later.
<br>
Planning graphs only work for propositional planning problems, hence we need to eliminate all variables by generating all possible substitutions.
<br>
For example, the planning graph of the `have_cake_and_eat_cake_too` problem might look like this
![title](images/cake_graph.jpg)
<br>
The black lines indicate links between states and actions.
<br>
In every planning problem, we are allowed to carry out the `no-op` action, ie, we can choose no action for a particular state.
These are called 'Persistence' actions and are represented in the graph by the small square boxes.
In technical terms, a persistence action has effects same as its preconditions.
This enables us to carry a state to the next level.
<br>
<br>
The gray lines indicate mutual exclusivity.
This means that the actions connected bya gray line cannot be taken together.
Mutual exclusivity (mutex) occurs in the following cases:
1. **Inconsistent effects**: One action negates the effect of the other. For example, _Eat(Cake)_ and the persistence of _Have(Cake)_ have inconsistent effects because they disagree on the effect _Have(Cake)_
2. **Interference**: One of the effects of an action is the negation of a precondition of the other. For example, _Eat(Cake)_ interferes with the persistence of _Have(Cake)_ by negating its precondition.
3. **Competing needs**: One of the preconditions of one action is mutually exclusive with a precondition of the other. For example, _Bake(Cake)_ and _Eat(Cake)_ are mutex because they compete on the value of the _Have(Cake)_ precondition.

In the module, planning graphs have been implemented using two classes, `Level` which stores data for a particular level and `Graph` which connects multiple levels together.
Let's look at the `Level` class.

In [50]:
psource(Level)

Each level stores the following data
1. The current state of the level in `current_state`
2. Links from an action to its preconditions in `current_action_links`
3. Links from a state to the possible actions in that state in `current_state_links`
4. Links from each action to its effects in `next_action_links`
5. Links from each possible next state from each action in `next_state_links`. This stores the same information as the `current_action_links` of the next level.
6. Mutex links in `mutex`.
<br>
<br>
The `find_mutex` method finds the mutex links according to the points given above.
<br>
The `build` method populates the data structures storing the state and action information.
Persistence actions for each clause in the current state are also defined here. 
The newly created persistence action has the same name as its state, prefixed with a 'P'.

Let's now look at the `Graph` class.

In [51]:
psource(Graph)

The class stores a problem definition in `pddl`, 
a knowledge base in `kb`, 
a list of `Level` objects in `levels` and 
all the possible arguments found in the initial state of the problem in `objects`.
<br>
The `expand_graph` method generates a new level of the graph.
This method is invoked when the goal conditions haven't been met in the current level or the actions that lead to it are mutually exclusive.
The `non_mutex_goals` method checks whether the goals in the current state are mutually exclusive.
<br>
<br>
Using these two classes, we can define a planning graph which can either be used to provide reliable heuristics for planning problems or used in the `GraphPlan` algorithm.
<br>
Let's have a look at the `GraphPlan` class.

In [52]:
psource(GraphPlan)

Given a planning problem defined as a PlanningProblem, `GraphPlan` creates a planning graph stored in `graph` and expands it till it reaches a state where all its required goals are present simultaneously without mutual exclusivity.
<br>
Once a goal is found, `extract_solution` is called.
This method recursively finds the path to a solution given a planning graph.
In the case where `extract_solution` fails to find a solution for a set of goals as a given level, we record the `(level, goals)` pair as a **no-good**.
Whenever `extract_solution` is called again with the same level and goals, we can find the recorded no-good and immediately return failure rather than searching again. 
No-goods are also used in the termination test.
<br>
The `check_leveloff` method checks if the planning graph for the problem has **levelled-off**, ie, it has the same states, actions and mutex pairs as the previous level.
If the graph has already levelled off and we haven't found a solution, there is no point expanding the graph, as it won't lead to anything new.
In such a case, we can declare that the planning problem is unsolvable with the given constraints.
<br>
<br>
To summarize, the `GraphPlan` algorithm calls `expand_graph` and tests whether it has reached the goal and if the goals are non-mutex.
<br>
If so, `extract_solution` is invoked which recursively reconstructs the solution from the planning graph.
<br>
If not, then we check if our graph has levelled off and continue if it hasn't.

Let's solve a few planning problems that we had defined earlier.

#### Air cargo problem
In accordance with the summary above, we have defined a helper function to carry out `GraphPlan` on the `air_cargo` problem.
The function is pretty straightforward.
Let's have a look.

In [53]:
psource(air_cargo_graphplan)

Let's instantiate the problem and find a solution using this helper function.

In [54]:
airCargoG = air_cargo_graphplan()
airCargoG

[[[Load(C2, P2, JFK),
   Fly(P2, JFK, SFO),
   Load(C1, P1, SFO),
   Fly(P1, SFO, JFK),
   PCargo(C1),
   PAirport(JFK),
   PPlane(P2),
   PAirport(SFO),
   PPlane(P1),
   PCargo(C2)],
  [Unload(C2, P2, SFO), Unload(C1, P1, JFK)]]]

Each element in the solution is a valid action.
The solution is separated into lists for each level.
The actions prefixed with a 'P' are persistence actions and can be ignored.
They simply carry certain states forward.
We have another helper function `linearize` that presents the solution in a more readable format, much like a total-order planner, but it is _not_ a total-order planner.

In [55]:
linearize(airCargoG)

[Load(C2, P2, JFK),
 Fly(P2, JFK, SFO),
 Load(C1, P1, SFO),
 Fly(P1, SFO, JFK),
 Unload(C2, P2, SFO),
 Unload(C1, P1, JFK)]

Indeed, this is a correct solution.
<br>
There are similar helper functions for some other planning problems.
<br>
Lets' try solving the spare tire problem.

In [56]:
spareTireG = spare_tire_graphplan()
linearize(spareTireG)

[Remove(Flat, Axle), Remove(Spare, Trunk), PutOn(Spare, Axle)]

Solution for the cake problem

In [57]:
cakeProblemG = have_cake_and_eat_cake_too_graphplan()
linearize(cakeProblemG)

[Eat(Cake), Bake(Cake)]

Solution for the Sussman's Anomaly configuration of three blocks.

In [58]:
sussmanAnomalyG = three_block_tower_graphplan()
linearize(sussmanAnomalyG)

[MoveToTable(C, A), Move(B, Table, C), Move(A, Table, B)]

Solution of the socks and shoes problem

In [59]:
socksShoesG = socks_and_shoes_graphplan()
linearize(socksShoesG)

[LeftSock, RightSock, LeftShoe, RightShoe]

### TOTAL ORDER PLANNER

In mathematical terminology, **total order**, **linear order** or **simple order** refers to a set *X* which is said to be totally ordered under &le; if the following statements hold for all *a*, *b* and *c* in *X*:
<br>
If *a* &le; *b* and *b* &le; *a*, then *a* = *b* (antisymmetry).
<br>
If *a* &le; *b* and *b* &le; *c*, then *a* &le; *c* (transitivity).
<br>
*a* &le; *b* or *b* &le; *a* (connex relation).

<br>
In simpler terms, a total order plan is a linear ordering of actions to be taken to reach the goal state.
There may be several different total-order plans for a particular goal depending on the problem.
<br>
<br>
In the module, the `Linearize` class solves problems using this paradigm.
At its core, the `Linearize` uses a solved planning graph from `GraphPlan` and finds a valid total-order solution for it.
Let's have a look at the class.

In [60]:
psource(Linearize)

The `filter` method removes the persistence actions (if any) from the planning graph representation.
<br>
The `orderlevel` method finds a valid total-ordering of a specified level of the planning-graph, given the state of the graph after the previous level.
<br>
The `execute` method sequentially calls `orderlevel` for all the levels in the planning-graph and returns the final total-order solution.
<br>
<br>
Let's look at some examples.

In [61]:
# total-order solution for air_cargo problem
Linearize(air_cargo()).execute()

[Load(C2, P2, JFK),
 Fly(P2, JFK, SFO),
 Load(C1, P1, SFO),
 Fly(P1, SFO, JFK),
 Unload(C2, P2, SFO),
 Unload(C1, P1, JFK)]

In [62]:
# total-order solution for spare_tire problem
Linearize(spare_tire()).execute()

[Remove(Flat, Axle), Remove(Spare, Trunk), PutOn(Spare, Axle)]

In [63]:
# total-order solution for three_block_tower problem
Linearize(three_block_tower()).execute()

[MoveToTable(C, A), Move(B, Table, C), Move(A, Table, B)]

In [64]:
# total-order solution for simple_blocks_world problem
Linearize(simple_blocks_world()).execute()

[ToTable(A, B), FromTable(B, A), FromTable(C, B)]

In [65]:
# total-order solution for socks_and_shoes problem
Linearize(socks_and_shoes()).execute()

[LeftSock, RightSock, LeftShoe, RightShoe]

### PARTIAL ORDER  PLANNER
A partial-order planning algorithm is significantly different from a total-order planner.
The way a partial-order plan works enables it to take advantage of _problem decomposition_ and work on each subproblem separately.
It works on several subgoals independently, solves them with several subplans, and then combines the plan.
<br>
A partial-order planner also follows the **least commitment** strategy, where it delays making choices for as long as possible.
Variables are not bound unless it is absolutely necessary and new actions are chosen only if the existing actions cannot fulfil the required precondition.
<br>
Any planning algorithm that can place two actions into a plan without specifying which comes first is called a **partial-order planner**.
A partial-order planner searches through the space of plans rather than the space of states, which makes it perform better for certain problems.
<br>
<br>
Let's have a look at the `PartialOrderPlanner` class.

In [66]:
psource(PartialOrderPlanner)

We will first describe the data-structures and helper methods used, followed by the algorithm used to find a partial-order plan.

Each plan has the following four components:

1. **`actions`**: a set of actions that make up the steps of the plan.
`actions` is always a subset of `pddl.actions` the set of possible actions for the given planning problem. 
The `start` and `finish` actions are dummy actions defined to bring uniformity to the problem. The `start` action has no preconditions and its effects constitute the initial state of the planning problem. 
The `finish` action has no effects and its preconditions constitute the goal state of the planning problem.
The empty plan consists of just these two dummy actions.
2. **`constraints`**: a set of temporal constraints that define the order of performing the actions relative to each other.
`constraints` does not define a linear ordering, rather it usually represents a directed graph which is also acyclic if the plan is consistent.
Each ordering is of the form A &lt; B, which reads as "A before B" and means that action A _must_ be executed sometime before action B, but not necessarily immediately before.
`constraints` stores these as a set of tuples `(Action(A), Action(B))` which is interpreted as given above.
A constraint cannot be added to `constraints` if it breaks the acyclicity of the existing graph.
3. **`causal_links`**: a set of causal-links. 
A causal link between two actions _A_ and _B_ in the plan is written as _A_ --_p_--> _B_ and is read as "A achieves p for B".
This imples that _p_ is an effect of _A_ and a precondition of _B_.
It also asserts that _p_ must remain true from the time of action _A_ to the time of action _B_.
Any violation of this rule is called a threat and must be resolved immediately by adding suitable ordering constraints.
`causal_links` stores this information as tuples `(Action(A), precondition(p), Action(B))` which is interpreted as given above.
Causal-links can also be called **protection-intervals**, because the link _A_ --_p_--> _B_ protects _p_ from being negated over the interval from _A_ to _B_.
4. **`agenda`**: a set of open-preconditions.
A precondition is open if it is not achieved by some action in the plan.
Planners will work to reduce the set of open preconditions to the empty set, without introducing a contradiction.
`agenda` stored this information as tuples `(precondition(p), Action(A))` where p is a precondition of the action A.

A **consistent plan** is a plan in which there are no cycles in the ordering constraints and no conflicts with the causal-links.
A consistent plan with no open preconditions is a **solution**.
<br>
<br>
Let's briefly glance over the helper functions before going into the actual algorithm.
<br>
**`expand_actions`**: generates all possible actions with variable bindings for use as a heuristic of selection of an open precondition.
<br>
**`find_open_precondition`**: finds a precondition from the agenda with the least number of actions that fulfil that precondition.
This heuristic helps form mandatory ordering constraints and causal-links to further simplify the problem and reduce the probability of encountering a threat.
<br>
**`find_action_for_precondition`**: finds an action that fulfils the given precondition along with the absolutely necessary variable bindings in accordance with the principle of _least commitment_.
In case of multiple possible actions, the action with the least number of effects is chosen to minimize the chances of encountering a threat.
<br>
**`cyclic`**: checks if a directed graph is cyclic.
<br>
**`add_const`**: adds `constraint` to `constraints` if the newly formed graph is acyclic and returns `constraints` otherwise.
<br>
**`is_a_threat`**: checks if the given `effect` negates the given `precondition`.
<br>
**`protect`**: checks if the given `action` poses a threat to the given `causal_link`.
If so, the threat is resolved by either promotion or demotion, whichever generates acyclic temporal constraints.
If neither promotion or demotion work, the chosen action is not the correct fit or the planning problem cannot be solved altogether.
<br>
**`convert`**: converts a graph from a list of edges to an `Action` : `set` mapping, for use in topological sorting.
<br>
**`toposort`**: a generator function that generates a topological ordering of a given graph as a list of sets.
Each set contains an action or several actions.
If a set has more that one action in it, it means that permutations between those actions also produce a valid plan.
<br>
**`display_plan`**: displays the `causal_links`, `constraints` and the partial order plan generated from `toposort`.
<br>

The **`execute`** method executes the algorithm, which is summarized below:
<br>
1. An open precondition is selected (a sub-goal that we want to achieve).
2. An action that fulfils the open precondition is chosen.
3. Temporal constraints are updated.
4. Existing causal links are protected. Protection is a method that checks if the causal links conflict
   and if they do, temporal constraints are added to fix the threats.
5. The set of open preconditions is updated.
6. Temporal constraints of the selected action and the next action are established.
7. A new causal link is added between the selected action and the owner of the open precondition.
8. The set of new causal links is checked for threats and if found, the threat is removed by either promotion or demotion.
   If promotion or demotion is unable to solve the problem, the planning problem cannot be solved with the current sequence of actions
   or it may not be solvable at all.
9. These steps are repeated until the set of open preconditions is empty.

A partial-order plan can be used to generate different valid total-order plans.
This step is called **linearization** of the partial-order plan.
All possible linearizations of a partial-order plan for `socks_and_shoes` looks like this.
<br>
![title](images/pop.jpg)
<br>
Linearization can be carried out in many ways, but the most efficient way is to represent the set of temporal constraints as a directed graph.
We can easily realize that the graph should also be acyclic as cycles in constraints means that the constraints are inconsistent.
This acyclicity is enforced by the `add_const` method, which adds a new constraint only if the acyclicity of the existing graph is not violated.
The `protect` method also checks for acyclicity of the newly-added temporal constraints to make a decision between promotion and demotion in case of a threat.
This property of a graph created from the temporal constraints of a valid partial-order plan allows us to use topological sort to order the constraints linearly.
A topological sort may produce several different valid solutions for a given directed acyclic graph.

Now that we know how `PartialOrderPlanner` works, let's solve a few problems using it.

In [67]:
st = spare_tire()
pop = PartialOrderPlanner(st)
pop.execute()

Causal Links
(Action(PutOn(Spare, Axle)), At(Spare, Axle), Action(Finish))
(Action(Start), Tire(Spare), Action(PutOn(Spare, Axle)))
(Action(Remove(Flat, Axle)), NotAt(Flat, Axle), Action(PutOn(Spare, Axle)))
(Action(Start), At(Flat, Axle), Action(Remove(Flat, Axle)))
(Action(Remove(Spare, Trunk)), At(Spare, Ground), Action(PutOn(Spare, Axle)))
(Action(Start), At(Spare, Trunk), Action(Remove(Spare, Trunk)))
(Action(Remove(Flat, Axle)), At(Flat, Ground), Action(Finish))

Constraints
Action(Start) < Action(Finish)
Action(Start) < Action(Remove(Spare, Trunk))
Action(Remove(Flat, Axle)) < Action(PutOn(Spare, Axle))
Action(Remove(Flat, Axle)) < Action(Finish)
Action(Remove(Spare, Trunk)) < Action(PutOn(Spare, Axle))
Action(Start) < Action(PutOn(Spare, Axle))
Action(Start) < Action(Remove(Flat, Axle))
Action(PutOn(Spare, Axle)) < Action(Finish)

Partial Order Plan
[{Action(Start)}, {Action(Remove(Flat, Axle)), Action(Remove(Spare, Trunk))}, {Action(PutOn(Spare, Axle))}, {Action(Finish)}]


We observe that in the given partial order plan, Remove(Flat, Axle) and Remove(Spare, Trunk) are in the same set.
This means that the order of performing these actions does not affect the final outcome.
That aside, we also see that the PutOn(Spare, Axle) action has to be performed after both the Remove actions are complete, which seems logically consistent.

In [68]:
sbw = simple_blocks_world()
pop = PartialOrderPlanner(sbw)
pop.execute()

Causal Links
(Action(FromTable(B, A)), On(B, A), Action(Finish))
(Action(FromTable(C, B)), On(C, B), Action(Finish))
(Action(Start), Clear(C), Action(FromTable(C, B)))
(Action(Start), Clear(A), Action(FromTable(B, A)))
(Action(Start), OnTable(C), Action(FromTable(C, B)))
(Action(Start), OnTable(B), Action(FromTable(B, A)))
(Action(ToTable(A, B)), Clear(B), Action(FromTable(C, B)))
(Action(Start), On(A, B), Action(ToTable(A, B)))
(Action(ToTable(A, B)), Clear(B), Action(FromTable(B, A)))
(Action(Start), Clear(A), Action(ToTable(A, B)))

Constraints
Action(Start) < Action(FromTable(B, A))
Action(Start) < Action(FromTable(C, B))
Action(Start) < Action(ToTable(A, B))
Action(ToTable(A, B)) < Action(FromTable(C, B))
Action(Start) < Action(Finish)
Action(ToTable(A, B)) < Action(FromTable(B, A))
Action(FromTable(C, B)) < Action(Finish)
Action(FromTable(B, A)) < Action(Finish)
Action(FromTable(B, A)) < Action(FromTable(C, B))

Partial Order Plan
[{Action(Start)}, {Action(ToTable(A, B))}, {Actio

We see that this plan does not have flexibility in selecting actions, ie, actions should be performed in this order and this order only, to successfully reach the goal state.

In [69]:
ss = socks_and_shoes()
pop = PartialOrderPlanner(ss)
pop.execute()

Causal Links
(Action(RightShoe), RightShoeOn, Action(Finish))
(Action(LeftShoe), LeftShoeOn, Action(Finish))
(Action(LeftSock), LeftSockOn, Action(LeftShoe))
(Action(RightSock), RightSockOn, Action(RightShoe))

Constraints
Action(Start) < Action(RightSock)
Action(Start) < Action(LeftSock)
Action(RightSock) < Action(RightShoe)
Action(RightShoe) < Action(Finish)
Action(Start) < Action(LeftShoe)
Action(LeftSock) < Action(LeftShoe)
Action(Start) < Action(RightShoe)
Action(Start) < Action(Finish)
Action(LeftShoe) < Action(Finish)

Partial Order Plan
[{Action(Start)}, {Action(LeftSock), Action(RightSock)}, {Action(RightShoe), Action(LeftShoe)}, {Action(Finish)}]


This plan again doesn't have constraints in selecting socks or shoes.
As long as both socks are worn before both shoes, we are fine.
Notice however, there is one valid solution,
<br>
LeftSock -> LeftShoe -> RightSock -> RightShoe
<br>
that the algorithm could not find as it cannot be represented as a general partially-ordered plan but is a specific total-order solution.

### Runtime differences
Let's briefly take a look at the running time of all the three algorithms on the `socks_and_shoes` problem.

In [70]:
ss = socks_and_shoes()

In [71]:
%%timeit
GraphPlan(ss).execute()

333 µs ± 8.86 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [72]:
%%timeit
Linearize(ss).execute()

1.29 ms ± 43.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [73]:
%%timeit
PartialOrderPlanner(ss).execute(display=False)

425 µs ± 17 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


We observe that `GraphPlan` is about 4 times faster than `Linearize` because `Linearize` essentially runs a `GraphPlan` subroutine under the hood and then carries out some transformations on the solved planning-graph.
<br>
We also find that `GraphPlan` is slightly faster than `PartialOrderPlanner`, but this is mainly due to the `expand_actions` method in `PartialOrderPlanner` that slows it down as it generates all possible permutations of actions and variable bindings.
<br>
Without heuristic functions, `PartialOrderPlanner` will be atleast as fast as `GraphPlan`, if not faster, but will have a higher tendency to encounter threats and conflicts which might take additional time to resolve.
<br>
Different planning algorithms work differently for different problems.

## PLANNING IN THE REAL WORLD
---
## PROBLEM
The `Problem` class is a wrapper for `PlanningProblem` with some additional functionality and data-structures to handle real-world planning problems that involve time and resource constraints.
The `Problem` class includes everything that the `PlanningProblem` class includes.
Additionally, it also includes the following attributes essential to define a real-world planning problem:
- a list of `jobs` to be done
- a dictionary of `resources`

It also overloads the `act` method to call the `do_action` method of the `HLA` class, 
and also includes a new method `refinements` that finds refinements or primitive actions for high level actions.
<br>
`hierarchical_search` and `angelic_search` are also built into the `Problem` class to solve such planning problems.

In [74]:
psource(Problem)

## HLA
To be able to model a real-world planning problem properly, it is  essential to be able to represent a _high-level action (HLA)_ that can be hierarchically reduced to primitive actions.

In [75]:
psource(HLA)

In addition to preconditions and effects, an object of the `HLA` class also stores:
- the `duration` of the HLA
- the quantity of consumption of _consumable_ resources
- the quantity of _reusable_ resources used
- a bool `completed` denoting if the `HLA` has been completed

The class also has some useful helper methods:
- `do_action`: checks if required consumable and reusable resources are available and if so, executes the action.
- `has_consumable_resource`: checks if there exists sufficient quantity of the required consumable resource.
- `has_usable_resource`: checks if reusable resources are available and not already engaged.
- `inorder`: ensures that all the jobs that had to be executed before the current one have been successfully executed.

## PLANNING PROBLEMS
---
## Job-shop Problem
This is a simple problem involving the assembly of two cars simultaneously.
The problem consists of two jobs, each of the form [`AddEngine`, `AddWheels`, `Inspect`] to be performed on two cars with different requirements and availability of resources.
<br>
Let's look at how the `job_shop_problem` has been defined on the  module.

In [76]:
psource(job_shop_problem)

The states of this problem are:
<br>
<br>
**Has(x, y)**: Car **'x'** _has_ **'y'** where **'y'** can be an Engine or a Wheel.

**~Has(x, y)**: Car **'x'** does _not have_ **'y'** where **'y'** can be an Engine or a Wheel.

**Inspected(c)**: Car **'c'** has been _inspected_.

**~Inspected(c)**: Car **'c'** has _not_ been inspected.

In the initial state, `C1` and `C2` are cars and neither have an engine or wheels and haven't been inspected.
`E1` and `E2` are engines.
`W1` and `W2` are wheels.
<br>
Our goal is to have engines and wheels on both cars and to get them inspected. We will discuss how to achieve this.
<br>
Let's define an object of the `job_shop_problem`.

In [77]:
jobShopProblem = job_shop_problem()

Before taking any actions, we will check if `jobShopProblem` has reached its goal.

In [78]:
print(jobShopProblem.goal_test())

False


We now define a possible solution that can help us reach the goal. 
The actions are then carried out on the `jobShopProblem` object.

The following actions are available to us:

**AddEngine1**: Adds an engine to the car C1. Takes 30 minutes to complete and uses an engine hoist.
 
**AddEngine2**: Adds an engine to the car C2. Takes 60 minutes to complete and uses an engine hoist.

**AddWheels1**: Adds wheels to car C1. Takes 30 minutes to complete. Uses a wheel station and consumes 20 lug nuts.

**AddWheels2**: Adds wheels to car C2. Takes 15 minutes to complete. Uses a wheel station and consumes 20 lug nuts as well.

**Inspect1**: Gets car C1 inspected. Requires 10 minutes of inspection by one inspector.

**Inspect2**: Gets car C2 inspected. Requires 10 minutes of inspection by one inspector.

In [79]:
solution = [jobShopProblem.jobs[1][0],
            jobShopProblem.jobs[1][1],
            jobShopProblem.jobs[1][2],
            jobShopProblem.jobs[0][0],
            jobShopProblem.jobs[0][1],
            jobShopProblem.jobs[0][2]]

for action in solution:
    jobShopProblem.act(action)

In [80]:
print(jobShopProblem.goal_test())

True


This is a valid solution and one of many correct ways to solve this problem.

## Double tennis problem
This problem is a simple case of a multiactor planning problem, where two agents act at once and can simultaneously change the current state of the problem. 
A correct plan is one that, if executed by the actors, achieves the goal.
In the true multiagent setting, of course, the agents may not agree to execute any particular plan, but atleast they will know what plans _would_ work if they _did_ agree to execute them.
<br>
In the double tennis problem, two actors A and B are playing together and can be in one of four locations: `LeftBaseLine`, `RightBaseLine`, `LeftNet` and `RightNet`.
The ball can be returned only if a player is in the right place.
Each action must include the actor as an argument.
<br>
Let's first look at the definition of the `double_tennis_problem` in the module.

In [81]:
psource(double_tennis_problem)

The states of this problem are:

**Approaching(Ball, loc)**: The `Ball` is approaching the location `loc`.

**Returned(Ball)**: One of the actors successfully hit the approaching ball from the correct location which caused it to return to the other side.

**At(actor, loc)**: `actor` is at location `loc`.

**~At(actor, loc)**: `actor` is _not_ at location `loc`.

Let's now define an object of `double_tennis_problem`.


In [82]:
doubleTennisProblem = double_tennis_problem()

Before taking any actions, we will check if `doubleTennisProblem` has reached the goal.

In [83]:
print(goal_test(doubleTennisProblem.goals, doubleTennisProblem.init))

False


As we can see, the goal hasn't been reached. 
We now define a possible solution that can help us reach the goal of having the ball returned.
The actions will then be carried out on the `doubleTennisProblem` object.

The actions available to us are the following:

**Hit(actor, ball, loc)**: returns an approaching ball if `actor` is present at the `loc` that the ball is approaching.

**Go(actor, to, loc)**: moves an `actor` from location `loc` to location `to`.

We notice something different in this problem though, 
which is quite unlike any other problem we have seen so far. 
The goal state of the problem contains a variable `a`.
This happens sometimes in multiagent planning problems 
and it means that it doesn't matter _which_ actor is at the `LeftNet` or the `RightNet`, as long as there is atleast one actor at either `LeftNet` or `RightNet`.

In [84]:
solution = [expr('Go(A, RightBaseLine, LeftBaseLine)'),
            expr('Hit(A, Ball, RightBaseLine)'),
            expr('Go(A, LeftNet, RightBaseLine)')]

for action in solution:
    doubleTennisProblem.act(action)

In [85]:
goal_test(doubleTennisProblem.goals, doubleTennisProblem.init)

True

It has now successfully reached its goal, ie, to return the approaching ball.